# Generische Programmierung mit Generics

#### Marcel Lüthi <br/> Departement Mathematik und Informatik

# Agenda

* Motivation
* Java Generics
* Type erasure und Rohtypen
* Typeinschränkungen
* Wildcards
* Generische Methoden

# Motivation

* Bisherige Lösung um Klassen für mehrere Typen verwendbar zu machen:
    * Nutzung von Supertyp Objekt

In [ ]:
class Stack { 
    
    private Object[] data = new Object[1000]; // Vereinfachung: Feste grösse
    private int nElements = 0; // Anzahl Elemente im Stack
 
    public void push(Object element) {
        data[nElements] = element;     
        nElements += 1;
    }
    
    public Object pop() { 
        if (nElements <= 0) {
            return null;
        }
        else {
            nElements -= 1;
            return data[nElements];
       }
    }
    public int size() { return this.nElements; }
}

# Probleme

* Wir brauchen explizite downcasts <br/> 
  ```Integer i = (Integer) stack.pop()```
* Intention nicht klar
    * Speichern wir Listen, Zahlen oder Strings?
* Nicht typesafe
    * Fehler zur Laufzeit - Keine Hilfe von Compiler

In [ ]:
Stack stack = new Stack();
stack.push("abc");
stack.push(5);

String s = (String) stack.pop();

* Klassen müssen in Hierarchie gebracht werden
    * Nicht möglich ohne Source Code

# Lösung: Java Generics

In [ ]:
class Stack<E> { 
    
    private E[] data = (E[])new Object[1000];
    private int nElements = 0; // Anzahl Elemente im Stack
 
 
    public void push(E element) {
        data[nElements] = element;     
        nElements += 1;
    }
    
    public E pop() { 
        if (nElements <= 0) {
            return null;
        }
        else {
            nElements -= 1;
            return data[nElements];
       }
    }
    public int size() { return this.nElements; }
}

# Nutzung des Generischen Stacks

In [ ]:
Stack<String> intStack = new Stack<String>();
intStack.push("abc");
String a = intStack.pop();


#### Miniübung
* Erzeugen Sie einen Stack von Strings
* Versuchen Sie ein Objekt vom falschen Typen auf den Stack zu legen

# Java Generics

### Alternative Syntax zur Erzeugung:

```Stack<Integer> intStack = new Stack<>();```
* Typ auf rechter Seite wird von Compiler erzeugt.

### Vorteile von Java Generics

* Homogene Datenstruktur mit Compilezeit Typprüfung
* Typsicherheit (keine Casts)
* Intention viel klarer.

# Terminologie

<div style="width:60%;margin-top:2cm">
<img src="images/generics-terminology.png"/>
</div>

# Mehrere Typparameter

* Wir können beliebig viele Typparameter einführen

#### Beispiel: Tuple Klasse

In [ ]:
class Tuple<T, U> {

    private T first;
    private U second;
    
    public Tuple(T first, U second) { 
        this.first = first;
        this.second = second;
    }
    public T getFirst() { return this.first;}
    public U getSecond() { return this.second; }
}

* Übung: Implementieren Sie die fehlenden Methoden

# Anwendung von Tuple

In [ ]:
Tuple<String, Date> t = new Tuple("Tom", new Date(1970, 12, 31));
System.out.println("the first element is: " +t.getFirst());
System.out.println("the second element is: " +t.getSecond());

# Type erasure und Rohtypen

# Type erasure

* Zur Kompilationszeit werden alle Typparameter ersetzt

![erasure](images./generics-erasure.png)

# Rohtypen

* Rohtypen können auch in Zuweisungen verwendet werden:

In [ ]:
Stack s = new Stack<Integer>();

Achtung!
```Stack``` ist nicht dasselbe wie ```Stack<Object>```

Das funktioniert

In [ ]:
Stack s = new Stack<Integer>();

aber das nicht:

In [ ]:
Stack<Object> s = new Stack<Integer>();

> Rohtypen sind nur aus Kompatibilitätsgründen vorhanden 
> 
> Sollten nie verwendet werden!

# Erzeugen von Objekten von Generischem Typ

> Nicht möglich!

In [ ]:
class Foo<T> {
    T t = new T();
}


* Keine Typinformation zur Laufzeit verfügbar.
    * Wie viel Speicher soll reserviert werden?

# Erzeugen von generischen Arrays

* Folgendes funktioniert nicht:

In [ ]:
class Foo<T> {
   T[] t = new T[100];
}

* Funktionierender Hack!

In [ ]:
class Foo<T> {
   T[] arrayOfTs = (T[]) new Object[100];
}

# Saubere Lösung: Factories

* Factory sind einfache Klassen um Objekte zu erzeugen.
* Saubere Alternative um Objekte von generischem Typ zu erzeugen
* Verbreitetes Muster in der Softwareentwicklung 

In [ ]:
interface Factory<T> { 
    public T createInstance();
}

# Factory Implementation

In [ ]:
class IntegerFactory implements Factory<Integer> { 
    int value = 0; 
    public IntegerFactory(int i)  {
        this.value = value;
    }
    public Integer createInstance() { 
        return new Integer(value);
    }
}

* Entsprechend auch für Strings, Dates, Integer[], ...

# Nutzung der Factory

In [ ]:
class Foo<T> {

    T t = null;
    
    public Foo(Factory<T> factory) { 
        t = factory.createInstance();
    }
}

In [ ]:
Foo<Integer> f = new Foo<>(new IntegerFactory(0));

# Übung

* Schreiben Sie eine Factory die Arrays von Integern erzeugt 
* Instanzieren sie die Klasse Foo (aus voriger Slide) für Integer Arrays.

In [ ]:
class IntegerArrayFactory implements Factory<Integer[]> {
    int size = 0;
    public IntegerArrayFactory(int size) { this.size = size; }
    
    public Integer[] createInstance() {
        return new Integer[size];
    }
}

new Foo(new IntegerArrayFactory(100))

# Typeinschränkungen 

# Typeinschränkungen (Bounded type parameters)

* Wir können Typeinschränkung auf generischen Parameter definieren
* Beispiel: Typ sollte vergleichbar sein: <br/>```class A<E extends Comparable>```
    * Nur Subtypen von Comparable können als generische Typen benutzt werden.     
* Mehrere Einschränkungen sind möglich
* Beispiel: Typ sollte vergleichbar und serialisierbar sein: <br/>```class A<E extends Comparable & Serializable>```

# Beispiel

In [ ]:
class SortedList <E extends Comparable> {
      E[] data = (E[]) new Comparable[1000];
      int nElements = 0;

      void add(E elem) {
          int i = nElements - 1;
          while (i >= 0 && elem.compareTo(data[i]) > 0) {  
              data[i+1] = data[i];
              i -= 1;  
          }
          data [i+1] = elem; 
          nElements++;          
     }
    
}

# Instanzierung 

Folgendes funktioniert:

In [ ]:
new SortedList<Integer>();
new SortedList<String>();
new SortedList<Comparable>();

Folgendes funktioniert nicht:

In [ ]:
new SortedList<Stack> ();
new SortedList<Object>();

Grund: ```Stack``` und  ```Object``` sind nicht ```Comparable```

# Vererbung

Einfach zu begreifen, wenn Sie folgendes verinnerlicht haben.

Für Klasse ```A<T>``` gilt:
* ```A``` steht für *Rohtyp*
    * Konkrete Klasse
* ```A<T>``` ist:
    * Konkrete Klasse für fixes T
    * Generische Klasse wenn T Typparameter ist

# Übung

In [ ]:
class Foo<T> {}

Welche dieser Statements funktioniert? Weshalb?

In [ ]:
class Bar<A> extends Foo {}

In [ ]:
class Bar<T> extends Foo<T> {}

In [ ]:
class Baz extends Foo<T> {}

# Übung

In [ ]:
class Foo {}
class Bar<T> extends Foo {}
class Baz<T, U> extends Foo {}

Welche dieser Statements funktioniert? Weshalb?

In [ ]:
Foo f = new Bar<Integer>()

In [ ]:
Foo g = new Baz<Integer, Float>()

# Übung

In [ ]:
class Foo<T> {}
class Bar<T> extends Foo<T> {}
class Baz<T, U> extends Foo<T> {}

Welche der folgenden Statements funktioniert? Weshalb?

In [ ]:
Foo<Integer> f = new Bar<Integer>();

In [ ]:
Foo<Integer> g = new Baz<Integer, Float>();

In [ ]:
Foo<Short> h = new Bar<Integer>();

# Überschreiben von Methoden

In [ ]:
class Foo<T> { 
    void f(T t) {}
}

* Wenn von konkreter Methode geerbt:
    * ```T``` wird durch konkreten Typ ersetzt

In [ ]:
class Bar extends Foo<Integer> { 
    void f(Integer i) {}
}

* Wenn von generischer Klasse geerbt
    * ```T``` bleibt Typparameter

In [ ]:
class Bar<T> extends Foo<T> { 
    void f(T t) {}
}

# Kovarianz, Kontravarianz und Invarianz

#### Kovarianz
Falls ```A``` ein Supertyp von ```B``` ist, dann ist ```Foo<A>``` ein Supertyp von ```Foo<B>```

#### Kontravarianz
Falls ```A``` ein Supertyp von ```B``` ist, dann ist ```Foo<A>``` ein Subtyp von ```Foo<B>```

#### Invarianz
Falls ```A``` ein Supertyp von ```B``` ist, dann ist ```Foo<A>``` weder ein Subtyp von ```Foo<B>``` noch ein Supertyp

> * Kontravarianz tritt nur in seltenen Fällen auf 
>    * Kann in diesen Kurs ignoriert werden

# Kovarianz und Invarianz

![images/variance.png](images/variance.png)

# Kovarianz und Invarianz in Java

Alle generischen Typen sind invariant

* Folgendes funktioniert nicht:

In [ ]:
class List<T> {}
List<Object> o = new List<String>();

> Grund: Kovariante Typen verkomplizieren das Typensystem enorm

# Kovarianz und Invarianz in Java

> Arrays sind kovariant in Java

* Folgendes funktioniert:

In [ ]:
Object[] objectArray = new String[100]

#### Grund

* Arrays gabe es bevor Generics eingeführt wurde
    * Viele nützliche Funktionen auf generischen Arrays könnten nicht programmiert werden.
    * Beispiel: ```void arrayEquals(object[] array1, object[] array2)```

# Wildcards

# Wildcards

> Gemeinsamer Basistyp für Generische Objekte
>    * Unabhängig von Parameter

```java 
List<?> l = new List<Integer>();
```

#### Beispiel

In [ ]:
class List<T> {}
List<?> l = null;
l = new List<Integer>();
l = new List<String>();
l = new List<Object>();

# Kompatibilitätsbeziehungen

```java
class A<T> {}
class B<T> extends A<T> {}
```

![wildcards-compatibility](images/wildcards-compatibility.png)


#### Beispiel

In [ ]:
class A<T> {}
class B<T> extends A<T> {}

Folgende Ausdrücke sind also alle gültig:

In [ ]:
A<?> a = new A<String>();
A<?> a = new B<String>();
A a = new A();
B b = new B<String>();
A a = new B<String>();
Object a = new A<String>();

# Bounded Wildcards

> Wildcard kann auf bestimmte Basisstypen eingeschränkt werden.

```java
Stack<? extends A> 
```

* Nur Objekte vom Typ ```A``` sollen gespeichert werden können.

# Bounded Wildcards: Beispiel

Ausgangslage:

In [ ]:
class Person {}
class Student extends Person {}
class Teacher extends Person {}

Stack<? extends Person> stack = null;

Das funktioniert

In [ ]:
stack = new Stack<Teacher>();
stack = new Stack<Student>();

aber das nicht

In [ ]:
stack = new Stack<Integer>();

# Limitierungen von Bounded Wildcards

* Bei Wildcards ist der genaue Typ nicht bekannt
    * Compiler muss gewisse Nutzung verbieten, um Typsicherheit zu gewährleisten

#### Ausgangslage:

```java
Stack<Teacher> teacherStack = new Stack<Teacher>();
teacherStack.push(new Teacher());
Stack<? extends Person> personStack = teacherStack;
```

* Von Stack entfernen mit ```stack.pop()``` funktioniert
    * Typ ist mindestens Person
* Hinzufügen geht nicht:
* ```Stack<? extends Person>``` könnte ```Stack<Teacher>``` oder ```Stack<Student>``` sein.

# Limitierungen von Bounded Wildcards

#### Beispiel

In [ ]:
Stack<Teacher> teacherStack = new Stack<Teacher>();
teacherStack.push(new Teacher());
Stack<? extends Person> personStack = teacherStack;

In [ ]:
Person p = personStack.pop();

In [ ]:
personStack.push(new Teacher());

# Generische Methoden

# Generische Methoden 

* Methoden, die mit unterschiedlichen Parametertypen arbeiten können
    * Syntax ```<T> void foo(T t) {}```
* Besprochene Regeln gelten auch für Methoden

#### Beispiel

In [ ]:
<T extends Comparable<T>> T max(T a, T b) {
    if (a.compareTo(b) > 1) {
        return a;
    } else { 
        return b;
    }
}

Nutzung:

In [ ]:
max(5, 7)

In [ ]:
max("abc", "cde")

# Generische Methoden: Typsicherheit

### Nicht Typsicher

* Deklaration von Methode mit gemeinsamen Supertyp.<br/>
```  Comparable max(Comparable a, Comparable b)```
* ```a``` muss nicht von demselben Typ wie b sein
* Drückt nicht aus was wir wollen


### Typesicher
* Deklaration mit Generics <br/>
``` <T extends Comparable> max(T a, T b) ```
* ```a``` und ```b``` haben denselben Typ
* Drückt genau aus was wir wollen

